In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

In [8]:
from langchain import OpenAI

openai = OpenAI(
    #model_name='text-davinci-003',
    model_name='text-curie-001',
    temperature=0.7
)

In [9]:
from langchain import PromptTemplate

template = """Answer the question based on the context below.
And act like a trading advisor with based on RSI and EMA indicators.

Context: The RSI and EMA indicator values are crucial when trading on a daily basis.
 RSI data indicating a decline can indicate an overbought condition between 70-100.
 Alse it shows overselling condition between 0-25.
 If lower period EMAs rise above higher period EMAs, it can be a bullish signal.
 For example, when the 5-day EMA crosses above the 10-day EMA.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [10]:
rsi_daily = 10
rsi_value = 30
ema_above = 7
ema_under = 13

print(openai(
    prompt_template.format(
        query=f"RSI_{rsi_daily} daily value is {rsi_value} and EMA_{ema_above} is above on EMA_{ema_under} day. "
              "What is your advice for this RSI and EMA?"
    )
))



The RSI indicator is indicating that the price is overbought and may be due for a correction. The EMA indicator is showing that the market is still relatively stable. While it is possible that the market will go down in the near future, it is also possible that the market will continue to rise.


In [11]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "RSI_10 daily value is 35 and EMA 5 daily value above than EMA 10 daily. What is your Advice for trading?",
        "answer": "The RSI value is near the overselling if youe are hodl any coin i think you continue to hold this coin." 
                "otherwise you can wait for the RSI decreasing for good buying oppurtunity and EMA value says bullish trend is continueing. You can think about bullish position."
    }, {
        "query": "RSI 10 daily value is 85 and EMA 7 daily value under than EMA 10 daily."
                "What is your Advice for trading?",
        "answer": "RSI value say this condition is overbought and you can sell any asset if you are hold any coin." 
                "Also EMA indicator shows a coin is in Bearish condition and you could sell if you are hold this coin."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are experts from conversations with an AI
trading advisor assistant.
The RSI and EMA indicator values are playing important role when trading on a daily basis.
 RSI data indicating a decline can indicate an overbought condition between 70-100. so you can sell your coin.
 Else it shows overselling condition between 0-25. so you can buy this coin.
 Overselling condition create a good oppurtunity for buyimg asset and overbought condition means that you should sell your asset.
 The EMA indicators generally says if lower daily period is
 above on higher daily period that means is bullish. if the condition is opposite for EMA than
 we can say that is bearish signal for selling assets.
Here are some examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [13]:
query = "RSI_10 daily value is 25 and EMA 6 daily value above than EMA 10 daily. What is your Advice for trading?"

print(openai(
    few_shot_prompt_template.format(query=query)
))



Since the RSI value is below the oversold territory, it may be a good time to sell your assets. Meanwhile, the EMA indicator suggests that the market is bullish, and so you may want to buy assets at this time.
